<a href="https://colab.research.google.com/github/Kirill-Mops/University/blob/main/Tasks/Ex_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [151]:
import time

def timer(fn):
  def wrapper(*args, **kwargs):
    st = time.perf_counter()
    result = fn(*args, **kwargs)
    dt = time.perf_counter() - st
    print(f"Время работы: {dt} сек")
    return result
  return wrapper

@timer
def first(n):
  d = []
  for i in range(n+1):
    if i % 2 == 0:
      d.append(i)
  return d

@timer
def second(n):
  return [x for x in range(n+1) if x % 2 == 0]

first(10);
second(10);


Время работы: 5.261999831418507e-06 сек
Время работы: 3.168999683111906e-06 сек


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [153]:
def cache_fib(fn):
  cache = [0, 1]
  def wrapper(n):
    if n == 0:
      return [0]
    while len(cache) <= n:
      cache.append(cache[-1] + cache[-2])
    return cache[:n+1]
  return wrapper


@cache_fib
def fib(n):
  if n == 0:
    return [0]
  s = [0, 1]
  for _ in range(2, n + 1):
    s.append(s[-1] + s[-2])
  return s[:n+1]

print(fib(5))

[0, 1, 1, 2, 3, 5]


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [157]:
import time

def timer(fn):
  def wrapper(*args, **kwargs):
    st = time.perf_counter()
    result = fn(*args, **kwargs)
    dt = time.perf_counter() - st
    print(f"Время работы: {dt} сек")
    return result
  return wrapper

def cache_fib(fn):
    cache = [0, 1]
    def wrapper(n):
        if n == 0:
            return [0]
        while len(cache) <= n:
            cache.append(cache[-1] + cache[-2])
        return cache[:n+1]
    return wrapper

def fib(n):
  if n == 0:
    return [0]
  s = [0, 1]
  for _ in range(2, n + 1):
    s.append(s[-1] + s[-2])
  return s[:n+1]

@timer
def first(n):
  fib(n)

@timer
@cache_fib
def second(n):
  fib(n)

first(6005);
second(6005);
second(6010);

Время работы: 0.0013268209986563306 сек
Время работы: 0.0012250170002516825 сек
Время работы: 2.3672999304835685e-05 сек


---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [27]:
def make_multiplier(n):
    def func(m):
      return n*m
    return func

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15

15


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [31]:
def rounder(n):
    def func(m):
      return round(m, n)
    return func

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14


3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [131]:
import time

def time_threshold(threshold):
    def decorator(fn):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = fn(*args, **kwargs)
            elapsed = time.time() - start
            if elapsed > threshold:
                print(f"{elapsed:.4f}")
            return result
        return wrapper
    return decorator


@time_threshold(threshold=0.5)
def long_running_function():
    time.sleep(0.8)

long_running_function()

0.8002


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [54]:
def my_generator(char, times):
    i = 1
    while i < times+1:
        yield char*i
        i += 1

for i in my_generator('*', 5):
    print(i)

*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [67]:
def infinite_sequence():
    i = 1
    while True:
      yield i
      i += 1

gen = infinite_sequence()

for _ in range(5):
    print(next(gen))

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [132]:
def combined_lists(lst1, lst2):
    for a, b in zip(lst1, lst2):
        yield a
        yield b

for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)

1
a
2
b
3
c


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [73]:
def reverse_string(string):
  for i in string[::-1]:
    yield i

for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [77]:
def powers_of_two(n):
  for i in range(n+1):
    yield 2**i

for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [135]:
def number_extractor(s):
  num = ''
  for ch in s:
    if ch.isdigit():
      num += ch
    elif num:
      yield int(num)
      num = ''
  if num:
    yield int(num)

for num in number_extractor('abc123def45gh6'):
    print(num)

123
45
6


---